In [5]:
import requests
import pandas as pd
import zipfile
import io
import re
from datetime import datetime

CONFIG_BUSQUEDA = {
    "Poblacion Total": ["", "Población total"],
    "PEA": ["", "Población económicamente activa (PEA)"], 
    "Desocupada": ["Población económicamente activa", "Desocupada"],
    "Sector Primario": ["3.2 Sector de actividad", "Primario"],
    "Sector Secundario": ["3.2 Sector de actividad", "Secundario"],
    "Sector Terciario": ["3.2 Sector de actividad", "Terciario"],
    "Educacion Sup": ["Nivel de instrucción", "Medio superior y superior"],
    "Informalidad TIL1": ["", "Tasa de informalidad laboral 1 (TIL1)"]
}

def procesar_enoe_test():
    print("⏳ [ENOE] Iniciando prueba de extracción...")
    
    base_url = "https://www.inegi.org.mx/contenidos/programas/enoe/15ymas/tabulados/"
    anio_actual = datetime.now().year
    url_final, anio_found, trim_found = None, None, None
    
    for a in [anio_actual, anio_actual-1]:
        if url_final: break
        for t in ["trim4", "trim3", "trim2", "trim1"]:
            test_url = f"{base_url}enoe_indicadores_estrategicos_{a}_{t}_xls.zip"
            try:
                r = requests.head(test_url, timeout=5)
                content_type = r.headers.get('Content-Type', '').lower()
                if r.status_code == 200 and ('zip' in content_type or 'octet-stream' in content_type):
                    url_final = test_url
                    anio_found = a
                    trim_found = t
                    print(f"   ✅ Archivo encontrado: {t.upper()} {a}")
                    break
            except: pass
            
    if not url_final:
        print("❌ No se encontró URL válida.")
        return None

    try:
        print(f"   📥 Descargando ZIP...")
        r = requests.get(url_final)
        z = zipfile.ZipFile(io.BytesIO(r.content))
        
        archivos = [f for f in z.namelist() if ("Entidades/" in f or "Nacional/" in f) and (f.endswith('.xlsx') or f.endswith('.xls'))]
        
        datos = []
        print(f"   📂 Procesando {len(archivos)} archivos (Estatales + Nacional)...")

        for arch in archivos:
            if "Nacional" in arch:
                estado = "Nacional"
            else:
                estado = arch.split("Entidad_")[-1].replace(".xlsx", "").replace(".xls", "").replace("_", " ").title()
            
            with z.open(arch) as f:
                df = pd.read_excel(f, header=None)
            
            registro = {'Estado': estado, 'Anio': anio_found, 'Trimestre': trim_found}
            
            col_val = 4
            for idx, row in df.iterrows():
                txt = " ".join([str(x).lower() for x in row[:5]])
                if "población total" in txt:
                    for c in range(4, min(15, len(row))):
                        try:
                            val = float(str(row[c]).replace(",","").replace(" ", ""))
                            if val > 1000: 
                                col_val = c
                                break
                        except: continue
                    break
            
            context = {i:"" for i in range(5)}
            sticky = {1:""}
            
            for idx, row in df.iterrows():
                cols = [str(row[i]).strip() if i < len(row) and pd.notna(row[i]) else "" for i in range(5)]
                
                indent = -1
                txt_row = ""
                
                for i, txt in enumerate(cols):
                    if txt:
                        indent = i; txt_row = txt; context[i] = txt
                        for j in range(i+1, 5): 
                            context[j] = ""
                            if j in sticky: sticky[j]=""
                        break
                
                if not txt_row: continue
                
                if indent == 1 and re.match(r'^(\d+\.?\d*)\s', txt_row): 
                    sticky[1] = txt_row
                
                path = [context[0]]
                if indent == 1 and not re.match(r'^(\d+\.?\d*)\s', txt_row) and sticky[1]:
                    path.append(sticky[1])
                    path.append(txt_row)
                else:
                    path.append(context[1])
                
                path += [context[k] for k in range(2,5)]
                
                path_str = " | ".join([p.lower() for p in path if p])
                
                for kpi, (padre, target) in CONFIG_BUSQUEDA.items():
                    if target.lower() in txt_row.lower():
                        if padre.lower() in path_str:
                            try: 
                                val_str = str(row[col_val]).replace(",","").replace(" ", "")
                                registro[kpi] = float(val_str)
                            except: pass
            
            datos.append(registro)

        df_final = pd.DataFrame(datos)
        return df_final
        
    except Exception as e:
        print(f"❌ Error crítico: {e}")
        return None

if __name__ == "__main__":
    df = procesar_enoe_test()
    
    if df is not None:
        print("\n--- RESULTADOS (Sectores Económicos) ---")
        cols_show = ['Estado', 'Sector Primario', 'Sector Secundario', 'Sector Terciario']
        print(df[cols_show].head())
        
        if 'Nacional' in df['Estado'].values:
            print("\n--- DATO NACIONAL ---")
            print(df[df['Estado'] == 'Nacional'][cols_show])
    else:
        print("Falló la prueba.")

⏳ [ENOE] Iniciando prueba de extracción...
   ✅ Archivo encontrado: TRIM3 2025
   📥 Descargando ZIP...
   📂 Procesando 33 archivos (Estatales + Nacional)...

--- RESULTADOS (Sectores Económicos) ---
                Estado  Sector Primario  Sector Secundario  Sector Terciario
0       Aguascalientes          27589.0           208332.0          412920.0
1  Baja California Sur          33035.0            90694.0          333315.0
2      Baja California          92805.0           578214.0         1006856.0
3             Campeche          68240.0            77146.0          275402.0
4              Chiapas         719364.0           378347.0         1180902.0

--- DATO NACIONAL ---
      Estado  Sector Primario  Sector Secundario  Sector Terciario
32  Nacional        6469941.0         14676108.0        37991032.0
